In [ ]:
import util
import plots
import numpy as np
import plan_params as pp
import pandas as pd
import inputs_for_mv
import ts_inputs as ts
import plotly.graph_objects as go
import plotly.express as px
import os

In [ ]:
output_df = ts.get_summarized_output('return_data.xlsx')
output_df

In [ ]:
#get data (policy weights, return, vol, corr, symbols) using inputs file
data_dict = util.get_data_dict(output_df)
data_dict.keys()

In [ ]:
#create plan object that has the relevant variables (cov, policy ret, var,fsv )
plan = pp.plan_params(data_dict['policy_weights'], data_dict['ret'], data_dict['vol'], data_dict['corr'], data_dict['symbols'])

In [ ]:
#TODO: import this data
bnds = pd.read_excel('C:\\Users\\NVG9HXP\\Documents\\Projects\\UPS_MV\\data\\'+'bounds.xlsx', sheet_name='bnds', index_col=0)
bnds =tuple(zip(bnds.Lower, bnds.Upper))
bnds

In [ ]:
cons = ({'type': 'ineq', 'fun': lambda x: np.sum(x[1:3]) - 0.5})

In [ ]:
# Optimizing for maximum sharpe ratio
opt_sharpe = plan.optimize(plan.min_sharpe_ratio, bnds, cons)
opt_sharpe

In [ ]:
# Optimizing for minimum variance
opt_var = plan.optimize(plan.min_variance,bnds, cons)
opt_var
min_ret = plan.portfolio_stats(opt_var['x'])[0]
min_ret

In [ ]:
# Optimizing for max return
opt_ret = plan.optimize(plan.min_ret,bnds, cons)
opt_ret
max_ret = np.around(plan.portfolio_stats(opt_ret['x']),4)[0]
max_ret

In [ ]:
#Getting data for efficient frontier portfolios
target_rets = np.linspace(min_ret,max_ret,100)
# target_rets = np.linspace(-.01,0.04,100)
t_vols = []
t_weights = []

for tr in target_rets:
    
    ef_cons = ({'type': 'eq', 'fun': lambda x: plan.portfolio_stats(x)[0] - tr},
               {'type': 'ineq', 'fun': lambda x: np.sum(x[1:3]) - 0.5},)
    
    opt_ef = plan.optimize(plan.min_volatility, bnds, ef_cons)
    
    t_vols.append(opt_ef['fun'])
    t_weights.append(opt_ef['x'])
    
target_vols = np.array(t_vols)
target_weights = np.array(t_weights)
optimized_weights = np.transpose(target_weights)
# opt_ef

In [ ]:
#Dataframe containing efficient frontier portfolios (return, vol, sharpe and weights)
df_ports = pd.DataFrame(np.column_stack([100*np.around(target_rets,6), 100*np.around(target_vols,6), np.around(target_rets/target_vols,6),100*np.around(target_weights,6)]),
                        columns=['Return', 'Volatility', 'Sharpe'] + plan.symbols)
df_ports.index.name = 'Portfolio'
df_ports['Portfolios'] = ['Portfolio']*100
df_ports['Portfolios'][df_ports[['Sharpe']].idxmax()[0]] = 'Max Sharpe Portfolio'
df_ports['Portfolios'][df_ports[['Volatility']].idxmin()[0]] = 'Min Var Portfolio'
df_ports

In [ ]:
#Asset Allocation Plot
combo_list = [('15+ STRIPS','forestgreen'),('Long Corporate','lightgreen'),('Equity','deeppink'),('Liquid Alternatives','blue'),
('Private Equity','darkred'),('Credit','yellow'),('Real Estate','orange'),('Cash','khaki')]

fig = go.Figure()

for combo in combo_list:
    fig.add_trace(go.Scatter(
     x= df_ports['Return'], y = df_ports[combo[0]],
     name = combo[0],
     mode = 'lines',
     line=dict(width=0.5, color=combo[1]),
     stackgroup = 'one'))
fig.update_layout(
        title = {'text':"<b>Asset Allocation</b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
        title_font_family = "Calibri",
        titlefont = {"size":24},
        xaxis = dict(ticksuffix="%"),
        yaxis = dict(ticksuffix="%"),
#         legend_title="<b>Assets</b>",
        plot_bgcolor='White'
    )
fig.update_xaxes(title_font_family = "Calibri",title_text = "<b>Returns</b>",
    title_font = {"size": 20},showline=True,linewidth=2,linecolor='black',mirror=False)
fig.update_yaxes(title_font_family = "Calibri",title_text = "<b>Weights</b>",range=(0,162),title_font = {"size": 20},
    showline=True,linewidth=2,linecolor='black',mirror=False)

fig.show()

In [ ]:
#Plotly version of the Efficient Frontier plot

fig = px.scatter(df_ports, x="Volatility", y="Return",color='Sharpe')
fig.update_layout(
    title={
            'text': "<b>Efficient Frontier Portfolio</b>",
            'y':0.9,
            'x':0.5,'xanchor': 'center',
            'yanchor': 'top'
            },
    title_font_family="Calibri",
    titlefont = {"size":20},
        xaxis = dict(tickfont = dict(size=14),ticksuffix="%"),
       yaxis = dict(ticksuffix="%"),
#     legend_title="Strategies",
    showlegend=True,
    plot_bgcolor='White'
                 )
fig.update_xaxes(title_font_family = "Calibri",title_text = "<b>Expected Volatility</b>",title_font = {"size": 20},
                    showline=True,linewidth=2,linecolor='black',mirror=False)

fig.update_yaxes(title_font_family = "Calibri",title_text = "<b>Expected Return</b>",title_font = {"size": 20},
                 showline=True,linewidth=2,linecolor='black',mirror=False)
fig.show()

In [ ]:
#Export Efficient Frontier portfoio data to excel
cwd = os.getcwd()
report_path = cwd+'\\reports\\'
file_path = report_path +'ef_portfolios.xlsx'
writer = pd.ExcelWriter(file_path,engine='xlsxwriter')
pd.DataFrame(np.column_stack([target_rets,target_vols,target_rets/target_vols,target_weights]),
                        columns=['Return', 'Volatility', 'Sharpe'] + plan.symbols).to_excel(writer,sheet_name='AA')
writer.save()